# 检索页及内容页
* 猎聘PC版 liepin.com 

## 生产用代码A

In [422]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession
import requests_html
from urllib.parse import urlparse, parse_qs, parse_qsl
from IPython.display import display, HTML
import os
import time
from random import random

# 基本输出入
fn = { "output":  {"path_raw": "data_raw",
                   "path_raw_content": "data_raw_content/{sn}.htm",
                   "path_data": "data_sets",
                   "datasets_xls": "data_sets/_content.xls",
                   "df_content_page": "data_sets/_content_page.tsv",
                   "dfl": "data_sets/_content_result.tsv",
                   "dfcr": "data_sets/dfcr.tsv",
                   "dfqr": "data_sets/dfqr.tsv",
                  } }


### 解析第一頁
* 基本页结构与连结: 页构连 dict_keys(['公司', '行业', '城市', '薪资', '更多', '页'])
* 建构参数: 参数  { "可调变量":可调变量, "可调变量框":可调变量框, "原始数据":原始数据, "参数模板":参数模板}

In [ ]:
# liepin第一页 页构连 liepin_search_page_params

def liepin_search_page_params(html):
# 函数：    
# 输入：requests_html 之 html物件，如r.html或 requests_html.HTML
# 输出：页构连 数据框字典，含 dict_keys(['公司', '行业', '城市', '薪资', '更多', '页'])各区块结构
    选 = html.xpath('//div[@data-selector="search-conditions"]/dl')    
    选d = { vs.xpath('//dt[@class="search-title"]/text()', first=True) \
            :vs.xpath('//dt//following-sibling::dd', first=True) \
            for vs in 选}

    ## 页面上方选择器
    页构连 = dict()
    for k, vs in 选d.items():
        变量名 = k.replace("：","")  
        #print(变量名)
        if '行业' in k:
            sel_main = vs.xpath('//ul/li')
            df = pd.DataFrame(  [ {"行业": v.xpath('//li/span', first=True).text, \
                                   "细分":link.text, \
                                   "href":link.xpath('//@href', first=True)} \
                                  for v in sel_main \
                                  for link in v.xpath('//li/div/a') ]   )   
        else:
            sel_C = vs.xpath('//a[not(img)]')
            df = pd.DataFrame(  [ {变量名:v.text, "href": v.xpath('//@href', first=True)} \
                                  for v in sel_C ]   )   
        页构连[变量名]=df          #display(df)

    ## 页面下方翻页器
    页 = html.xpath('//div[@class="pagerbar"]/a')
    if len(页)!=0:
        页d = [ { "页": p.text, "href" :p.xpath('//@href', first=True)} for p in  页 ]
        页构连["页"] = pd.DataFrame(页d)
    else:
        页构连 = None 
    return(页构连)

In [115]:
# 第一页访问及收集
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
r = session.get( url )
    
# 基本页结构与连结
html = r.html # html 物件
页构连 = liepin_search_page_params(html)
print(页构连.keys())

dict_keys(['公司', '行业', '城市', '薪资', '更多', '页'])


In [118]:
# 建构参数：找到关键参数及参数结构

def parse_url_qs_list (url_list):
# 函数： 
# 输入：url列表 url_list
# 输出：url各别解析成果，六大块
    out_list = []
    for url in url_list:
        #print (url)
        six_parts = urlparse(url) 
        out = parse_qs(six_parts.query)
        out_list.append(out)
    return (out_list[0])

def liepin_search_params(页构连):
    参数 = dict()
    if 页构连 is None:
        return(参数)
    #key = '页'#'行业'  # '公司', '行业', '城市', '薪资', '更多', '页'
    for key in 页构连.keys():
        df = 页构连[key]

        if 'href' in df.columns: # 必需要有超连结才处理，若无则无法就 url取参数
            df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['href'] ])  # 解析 query
            df = df.join(df_qs)
            #display (df_qs)
            df_qs_variables = df_qs.nunique()
            qs_dict      = df_qs_variables[df_qs_variables >=2].to_dict()
            qs_dict_same = df_qs_variables[df_qs_variables <=1].to_dict()

            #for k,v in qs_dict.items():
            #    print (k, df_qs[k].unique())

            原始数据=df
            可调变量 = list(qs_dict.keys())

            if len(可调变量)==0: # 无可调, 如不能翻页
                可调变量框 = None
                dt = None
                可调变量反查 = None
                参数模板 = None
            else:
                if key == '行业':
                    可调变量框 = 原始数据.drop(list(qs_dict_same.keys()), axis=1).dropna().melt([key,'细分'])
                    dt = 可调变量框.set_index(["variable",key,'细分']).loc[可调变量].unstack('variable')
                    dt.columns = dt.columns.get_level_values(1)
                else:
                    可调变量框 = 原始数据.drop(list(qs_dict_same.keys()), axis=1).dropna().melt(key)
                    dt = 可调变量框.set_index(["variable",key]).loc[可调变量].unstack('variable')
                    dt.columns = dt.columns.get_level_values(1)

                可调变量反查 = dt.to_dict()
                href_列表 = [x for x in 原始数据["href"].to_list() if "javascript" not in x]
                参数模板 = parse_url_qs_list(href_列表)
            # display(dt)        
            out = { "可调变量":可调变量, "可调变量框":可调变量框, "可调变量反查框":dt, "可调变量反查":可调变量反查, "原始数据":原始数据, "参数模板":参数模板}
        else:
            out = { "可调变量":None, "可调变量框":None, "可调变量反查框":None, "可调变量反查":None, "原始数据":None, "参数模板":None}
        参数[key] = out
    return(参数)

参数 = liepin_search_params(页构连)
参数['页']["参数模板"] # 参数['行业']["原始数据"]
# display(参数)

{'init': ['-1'],
 'headckid': ['b5f20ffd3c660405'],
 'fromSearchBtn': ['2'],
 'ckid': ['b5f20ffd3c660405°radeFlag=0'],
 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
 'd_sfrom': ['search_unknown'],
 'd_ckId': ['4210c2dce2e2ada96519cfff4903b223'],
 'd_curPage': ['0'],
 'd_pageSize': ['40'],
 'd_headId': ['4210c2dce2e2ada96519cfff4903b223'],
 'curPage': ['1']}

### 建构查询框

In [5]:
# 建构查询关键字列表 list_key
src = '''2019年互联网新兴设计人才白皮书
本次白皮书的主要研究对象为互联网新兴设计，主要指以应用互联网技术为特征，
基于人本主义思想，关注用户体验丶交互过程，服务于数字化产品及服务的研发丶
运营丶推广的综合性设计。白皮书中主要抽取各大招聘网站中与互联网新兴设计
相关的职位数据进行分析，如界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶
体验设计丶服务设计丶信息设计丶全栈/全链路设计等。
'''

list_key = [[x.strip()] if "/" not in x else [x.split("/")[0]+"设计", x.split("/")[1]]  \
            for x in src.split("如")[1].split("等")[0].split("丶") ]
list_key =  [item for sublist in list_key for item in sublist] # flattenlist
print(list_key)

['界面设计', 'UI设计', '视觉设计', '交互设计', '用户体验', '体验设计', '服务设计', '信息设计', '全栈设计', '全链路设计']


In [6]:
# 建构查询参数计划框架 dfq
参数_key = '公司'
可调变量 = list(参数[参数_key]["可调变量"])

可调变量框 = 参数[参数_key]["可调变量框"]
可调变量反查 = 参数[参数_key]["可调变量反查"]

payload_template = 参数[参数_key]["参数模板"].copy()  # 复制一份

records = list()
for tag, vs in 可调变量反查.items():
    print (tag)
    for tag_display, tag_code in vs.items():
        print ("-",tag_display,tag_code)
        print (" -",end="")
        for i, key in enumerate(list_key):
            print (key, end="丶")
            payload = payload_template.copy()
            payload.update ({ 'key': [ key ], tag: [tag_code]})
                                # 查询关键词key 查询可调变量tag 输入 参数模板payload_template
            records.append({'tag':tag,'tag_display':tag_display,'tag_code':tag_code,\
                            'key':key, 'keyi':i, 'payload':payload})
        print()
        
dfq = pd.DataFrame(records)
dfq = dfq.assign( 
          fn =  lambda x : x["tag"]+"_"+ x["tag_code"] +"-"+ "key" +"_"+ x["keyi"].astype("str"), 
          fn_full = lambda x : x["tag"] +"_"+ x["tag_code"] +"_"+  x["tag_display"] +"-"+ \
                  "key" +"_"+ x["keyi"].astype("str") +"_"+  x["key"] \
          )
dfq = dfq.assign(
            fn_tsv = lambda x : [ os.path.join ( fn["output"]["path_data"], ".".join( [ _x_, "tsv"] ) ) for _x_ in x["fn_full"]],
            fn_htm = lambda x : [ os.path.join ( fn["output"]["path_raw"], ".".join( [ _x_, "htm"] ) ) for _x_ in x["fn"]],
        )
dfq

compTag
- 2018互联网300强 182
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶
- AI创新成长50强 189
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶
- 上市公司 156
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶
- 中国500强 155
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶
- 制造业500强 186
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶
- 独角兽 130
 -界面设计丶UI设计丶视觉设计丶交互设计丶用户体验丶体验设计丶服务设计丶信息设计丶全栈设计丶全链路设计丶


tag  tag_display tag_code    key  keyi  \
0   compTag  2018互联网300强      182   界面设计     0   
1   compTag  2018互联网300强      182   UI设计     1   
2   compTag  2018互联网300强      182   视觉设计     2   
3   compTag  2018互联网300强      182   交互设计     3   
4   compTag  2018互联网300强      182   用户体验     4   
5   compTag  2018互联网300强      182   体验设计     5   
6   compTag  2018互联网300强      182   服务设计     6   
7   compTag  2018互联网300强      182   信息设计     7   
8   compTag  2018互联网300强      182   全栈设计     8   
9   compTag  2018互联网300强      182  全链路设计     9   
10  compTag    AI创新成长50强      189   界面设计     0   
11  compTag    AI创新成长50强      189   UI设计     1   
12  compTag    AI创新成长50强      189   视觉设计     2   
13  compTag    AI创新成长50强      189   交互设计     3   
14  compTag    AI创新成长50强      189   用户体验     4   
15  compTag    AI创新成长50强      189   体验设计     5   
16  compTag    AI创新成长50强      189   服务设计     6   
17  compTag    AI创新成长50强      189   信息设计     7   
18  compTag    AI创新成长50强      189   全栈设计     8   
19  compTag    AI创新成长50强      189  全链路设计     9   
20  compTag         上市公司      156   界面设计     0   
21  compTag         上市公司      156   UI设计     1   
22  compTag         上市公司      156   视觉设计     2   
23  compTag         上市公司      156   交互设计     3   
24  compTag         上市公司      156   用户体验     4   
25  compTag         上市公司      156   体验设计     5   
26  compTag         上市公司      156   服务设计     6   
27  compTag         上市公司      156   信息设计     7   
28  compTag         上市公司      156   全栈设计     8   
29  compTag         上市公司      156  全链路设计     9   
30  compTag       中国500强      155   界面设计     0   
31  compTag       中国500强      155   UI设计     1   
32  compTag       中国500强      155   视觉设计     2   
33  compTag       中国500强      155   交互设计     3   
34  compTag       中国500强      155   用户体验     4   
35  compTag       中国500强      155   体验设计     5   
36  compTag       中国500强      155   服务设计     6   
37  compTag       中国500强      155   信息设计     7   
38  compTag       中国500强      155   全栈设计     8   
39  compTag       中国500强      155  全链路设计     9   
40  compTag      制造业500强      186   界面设计     0   
41  compTag      制造业500强      186   UI设计     1   
42  compTag      制造业500强      186   视觉设计     2   
43  compTag      制造业500强      186   交互设计     3   
44  compTag      制造业500强      186   用户体验     4   
45  compTag      制造业500强      186   体验设计     5   
46  compTag      制造业500强      186   服务设计     6   
47  compTag      制造业500强      186   信息设计     7   
48  compTag      制造业500强      186   全栈设计     8   
49  compTag      制造业500强      186  全链路设计     9   
50  compTag          独角兽      130   界面设计     0   
51  compTag          独角兽      130   UI设计     1   
52  compTag          独角兽      130   视觉设计     2   
53  compTag          独角兽      130   交互设计     3   
54  compTag          独角兽      130   用户体验     4   
55  compTag          独角兽      130   体验设计     5   
56  compTag          独角兽      130   服务设计     6   
57  compTag          独角兽      130   信息设计     7   
58  compTag          独角兽      130   全栈设计     8   
59  compTag          独角兽      130  全链路设计     9   

                                              payload                 fn  \
0   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_0   
1   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_1   
2   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_2   
3   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_3   
4   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_4   
5   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_5   
6   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_6   
7   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_7   
8   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_8   
9   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_9   
10  {'init': ['-1'], 'compTag': ['189'], 'ckid': [...  compTag_189-key_0   
11  {'init': ['-1'], 'compTag': ['189'], 'cki

### 解析查询结果页

In [9]:
# requests_liepin查询首页存放与解析

def requests_liepin( url, payload):
    
    数据, raw = "", ""
    
    try:
        r = session.get( url , params = payload)
        raw = r.html.raw_html

        主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')
        dict_xpaths={ 
            'text': {
                'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
                '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
                '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
                '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
                '职称':    '//div[contains(@class,"job-info")]/h3/a', 
                '公司地点': '//div[contains(@class,"job-info")]/p/a',
                '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
            },
            'text_content': {
            },
            'href': {
                '链结':    '//div[contains(@class,"job-info")]/h3/a', 
                '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
            }
        }

        def get_e_text_content(_xpath_):
            暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
            return(暂存结果)

        def get_e_text(_xpath_):
            暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
            return(暂存结果)

        def get_e_href(_xpath_):
            暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                       if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                       else "" for e in 主要元素]
            return(暂存结果)

        数据字典 = dict()
        数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
        数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
        数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

        数据 = pd.DataFrame(数据字典)
    except:
        pass
    
    return (数据, raw)

### 用查询框執行批量抓取

In [174]:
# 用查询框dfq批量抓取, tsv raw的暂存, 并把抓取状态及原始htm檔存入dfqr, 和dfq一一对映i
records_crawl = []

url = r.url

for i in dfq.index:
    payload = dfq.loc[i]['payload']
    fn_tsv = dfq.loc[i]['fn_tsv']
    fn_htm = dfq.loc[i]['fn_htm']
    tag_display = dfq.loc[i]['tag_display']
    key = dfq.loc[i]['key']

    df_tsv, htm_raw =  requests_liepin(url, payload)   # 抓
    df_tsv = df_tsv.assign (tag_display = tag_display)  # 区分  tag_display    
    df_tsv.index.name = "rank"
    df_tsv = df_tsv.reset_index()
    df_tsv = df_tsv.assign (i = i)
    df_tsv = df_tsv.assign (rank = lambda x: x["rank"] +1)
    df_tsv = df_tsv.assign (keyword = key)              # 区分  keyword    

    df_tsv.to_csv(fn_tsv, encoding="utf8", sep="\t")  # 存 tsv
    with open(fn_htm, "wb") as f:                     # 存 htm 
        f.write(htm_raw)

    stats_code = r.status_code # http code，记录检查用
    size_tsv = len(df_tsv)  # tsv大小，记录检查用
    size_htm = len(htm_raw) # htm大小，记录检查用
    
    _record_ = {"i":i, "size_tsv":size_tsv, "size_htm":size_htm}
    print ('抓',tag_display,key,_record_)
    records_crawl.append(_record_)
    
    time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
    
dfqr = pd.DataFrame(records_crawl)

抓 2018互联网300强 界面设计 {'i': 0, 'size_tsv': 40, 'size_htm': 222716}
抓 2018互联网300强 UI设计 {'i': 1, 'size_tsv': 40, 'size_htm': 220255}
抓 2018互联网300强 视觉设计 {'i': 2, 'size_tsv': 40, 'size_htm': 219747}
抓 2018互联网300强 交互设计 {'i': 3, 'size_tsv': 40, 'size_htm': 220214}
抓 2018互联网300强 用户体验 {'i': 4, 'size_tsv': 40, 'size_htm': 221979}
抓 2018互联网300强 体验设计 {'i': 5, 'size_tsv': 40, 'size_htm': 220192}
抓 2018互联网300强 服务设计 {'i': 6, 'size_tsv': 40, 'size_htm': 224171}
抓 2018互联网300强 信息设计 {'i': 7, 'size_tsv': 40, 'size_htm': 223136}
抓 2018互联网300强 全栈设计 {'i': 8, 'size_tsv': 8, 'size_htm': 120803}
抓 2018互联网300强 全链路设计 {'i': 9, 'size_tsv': 40, 'size_htm': 203111}
抓 AI创新成长50强 界面设计 {'i': 10, 'size_tsv': 40, 'size_htm': 231338}
抓 AI创新成长50强 UI设计 {'i': 11, 'size_tsv': 40, 'size_htm': 228808}
抓 AI创新成长50强 视觉设计 {'i': 12, 'size_tsv': 40, 'size_htm': 225846}
抓 AI创新成长50强 交互设计 {'i': 13, 'size_tsv': 40, 'size_htm': 236704}
抓 AI创新成长50强 用户体验 {'i': 14, 'size_tsv': 40, 'size_htm': 235815}
抓 AI创新成长50强 体验设计 {'i': 15, 'size_tsv': 40, 's

In [389]:
dfqr.to_csv(fn["output"]["dfqr"], encoding="utf8", sep="\t")

In [177]:
# dfqr 檢查
dfqr.T

0       1       2       3       4       5       6       7   \
i              0       1       2       3       4       5       6       7   
size_tsv      40      40      40      40      40      40      40      40   
size_htm  222716  220255  219747  220214  221979  220192  224171  223136   

              8       9   ...      50      51      52      53      54      55  \
i              8       9  ...      50      51      52      53      54      55   
size_tsv       8      40  ...      40      40      40      40      40      40   
size_htm  120803  203111  ...  237250  234241  236194  236125  236334  236217   

              56      57      58      59  
i             56      57      58      59  
size_tsv      40      40       3      18  
size_htm  233108  235828  105682  150053  

[3 rows x 60 columns]

## 生产用代码B

### 翻页

In [178]:
fn_htm = "data_raw/compTag_130-key_8.htm"
with open (fn_htm, encoding = "utf8", mode = "r") as fp:
        html_load = fp.read()                  #html_load

html_load = requests_html.HTML(html= html_load, url='https://localhost/')
_页构连_ = liepin_search_page_params(html_load)   #html_load
参数 = liepin_search_params(_页构连_)
参数['页']['可调变量']==[]

True

In [179]:
dfq

tag  tag_display tag_code    key  keyi  \
0   compTag  2018互联网300强      182   界面设计     0   
1   compTag  2018互联网300强      182   UI设计     1   
2   compTag  2018互联网300强      182   视觉设计     2   
3   compTag  2018互联网300强      182   交互设计     3   
4   compTag  2018互联网300强      182   用户体验     4   
5   compTag  2018互联网300强      182   体验设计     5   
6   compTag  2018互联网300强      182   服务设计     6   
7   compTag  2018互联网300强      182   信息设计     7   
8   compTag  2018互联网300强      182   全栈设计     8   
9   compTag  2018互联网300强      182  全链路设计     9   
10  compTag    AI创新成长50强      189   界面设计     0   
11  compTag    AI创新成长50强      189   UI设计     1   
12  compTag    AI创新成长50强      189   视觉设计     2   
13  compTag    AI创新成长50强      189   交互设计     3   
14  compTag    AI创新成长50强      189   用户体验     4   
15  compTag    AI创新成长50强      189   体验设计     5   
16  compTag    AI创新成长50强      189   服务设计     6   
17  compTag    AI创新成长50强      189   信息设计     7   
18  compTag    AI创新成长50强      189   全栈设计     8   
19  compTag    AI创新成长50强      189  全链路设计     9   
20  compTag         上市公司      156   界面设计     0   
21  compTag         上市公司      156   UI设计     1   
22  compTag         上市公司      156   视觉设计     2   
23  compTag         上市公司      156   交互设计     3   
24  compTag         上市公司      156   用户体验     4   
25  compTag         上市公司      156   体验设计     5   
26  compTag         上市公司      156   服务设计     6   
27  compTag         上市公司      156   信息设计     7   
28  compTag         上市公司      156   全栈设计     8   
29  compTag         上市公司      156  全链路设计     9   
30  compTag       中国500强      155   界面设计     0   
31  compTag       中国500强      155   UI设计     1   
32  compTag       中国500强      155   视觉设计     2   
33  compTag       中国500强      155   交互设计     3   
34  compTag       中国500强      155   用户体验     4   
35  compTag       中国500强      155   体验设计     5   
36  compTag       中国500强      155   服务设计     6   
37  compTag       中国500强      155   信息设计     7   
38  compTag       中国500强      155   全栈设计     8   
39  compTag       中国500强      155  全链路设计     9   
40  compTag      制造业500强      186   界面设计     0   
41  compTag      制造业500强      186   UI设计     1   
42  compTag      制造业500强      186   视觉设计     2   
43  compTag      制造业500强      186   交互设计     3   
44  compTag      制造业500强      186   用户体验     4   
45  compTag      制造业500强      186   体验设计     5   
46  compTag      制造业500强      186   服务设计     6   
47  compTag      制造业500强      186   信息设计     7   
48  compTag      制造业500强      186   全栈设计     8   
49  compTag      制造业500强      186  全链路设计     9   
50  compTag          独角兽      130   界面设计     0   
51  compTag          独角兽      130   UI设计     1   
52  compTag          独角兽      130   视觉设计     2   
53  compTag          独角兽      130   交互设计     3   
54  compTag          独角兽      130   用户体验     4   
55  compTag          独角兽      130   体验设计     5   
56  compTag          独角兽      130   服务设计     6   
57  compTag          独角兽      130   信息设计     7   
58  compTag          独角兽      130   全栈设计     8   
59  compTag          独角兽      130  全链路设计     9   

                                              payload                 fn  \
0   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_0   
1   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_1   
2   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_2   
3   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_3   
4   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_4   
5   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_5   
6   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_6   
7   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_7   
8   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_8   
9   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...  compTag_182-key_9   
10  {'init': ['-1'], 'compTag': ['189'], 'ckid': [...  compTag_189-key_0   
11  {'init': ['-1'], 'compTag': ['189'], 'cki

In [182]:
def find_pp_max(参数):
    pp = 参数['页']['可调变量反查框']
    pp = pp [ pd.to_numeric(pp.curPage, errors='coerce').notnull() ]
    pp = pp.assign(curpage = lambda x: x["curPage"].astype(int))
    长度 = pp.curpage.max()+1
    return(长度)

_temp_out_ = list()

for i in dfq.index:
    fn_htm = dfq.loc[i]["fn_htm"]
    payload_template = dfq.loc[i]["payload"]
    dfqpp_slice = dfq.loc[[i]].copy()

    with open (fn_htm, encoding = "utf8", mode = "r") as fp:
        html_load = fp.read()                  #html_load
    html = requests_html.HTML(html= html_load, url='https://localhost/')

    _页构连_ = liepin_search_page_params(html)   #html_load
    
    if _页构连_ is None:
        dfqpp = dfqpp_slice
        _tag_ = 'curPage'
        pp = 0
        payload = payload_template.copy()
        payload[_tag_] = [ str(pp) ]
        dfqpp_slice.loc[:,_tag_] = [ str(pp) ]
        dfqpp_slice.loc[:,"pp"] = [ str(pp) ]
        dfqpp_slice.loc[:,"payload"] = [ payload ]
        _temp_ = [dfqpp_slice]
        dfqpp = pd.concat(_temp_).reset_index(drop=True)
        
    else:
        参数 = liepin_search_params(_页构连_)
        if (参数['页']['可调变量']!=[] ):
            payload_template = 参数['页']["参数模板"].copy()
            可调变量反查 = 参数['页']['可调变量反查']

            _tag_ = list(可调变量反查.keys())[0]

            pp_max = find_pp_max(参数)
            pp_max2 = max( [int(x) for x in list(list(可调变量反查.values())[0].values())]  )  + 1
        else:
            pp_max = 0 + 1
            pp_max2 = 0 + 1
            
        _temp_ = list()
        for pp in range(0,pp_max):
            payload = payload_template.copy()
            payload[_tag_] = [ str(pp) ]

            dfqpp_slice.loc[:,_tag_] = [ str(pp) ]
            dfqpp_slice.loc[:,"pp"] = [ str(pp) ]
            dfqpp_slice.loc[:,"payload"] = [ payload ]
            #display(dfqpp_slice)
            _temp_.append(dfqpp_slice.copy())

            
        print(fn_htm, pp_max, pp_max2)

        ## _temp_
        dfqpp = pd.concat(_temp_).reset_index(drop=True)

    dfqpp['i'] = i

    #dfqpp = dfqpp.set_index(pd.Index(dfqpp[["i","pp"]]))
    dfqpp = dfqpp.assign( 
              fn =  lambda x : x["tag"]+"_"+ x["tag_code"] +"-"+ "key" +"_"+ x["keyi"].astype("str") +"-"+ "pp" +"_"+ x["pp"].astype("str"), 
              fn_full = lambda x : x["tag"] +"_"+ x["tag_code"] +"_"+  x["tag_display"] +"-"+ \
                      "key" +"_"+ x["keyi"].astype("str") +"_"+  x["key"] +"-"+  \
                      "pp" +"_"+ x["pp"].astype("str")  
              )
    dfqpp = dfqpp.assign(
                fn_tsv = lambda x : [ os.path.join ( fn["output"]["path_data"], ".".join( [ _x_, "tsv"] ) ) for _x_ in x["fn_full"]],
                fn_htm = lambda x : [ os.path.join ( fn["output"]["path_raw"], ".".join( [ _x_, "htm"] ) ) for _x_ in x["fn"]],
            )
    _temp_out_.append(dfqpp)
    
dfqpp = pd.concat(_temp_out_).reset_index(drop=True)

data_raw\compTag_182-key_0.htm 10 10
data_raw\compTag_182-key_1.htm 10 10
data_raw\compTag_182-key_2.htm 10 10
data_raw\compTag_182-key_3.htm 10 10
data_raw\compTag_182-key_4.htm 10 10
data_raw\compTag_182-key_5.htm 10 10
data_raw\compTag_182-key_6.htm 10 10
data_raw\compTag_182-key_7.htm 7 7
data_raw\compTag_182-key_8.htm 1 1
data_raw\compTag_182-key_9.htm 1 1
data_raw\compTag_189-key_0.htm 5 5
data_raw\compTag_189-key_1.htm 5 5
data_raw\compTag_189-key_2.htm 3 3
data_raw\compTag_189-key_3.htm 10 10
data_raw\compTag_189-key_4.htm 10 10
data_raw\compTag_189-key_5.htm 10 10
data_raw\compTag_189-key_6.htm 1 1
data_raw\compTag_189-key_7.htm 1 1
data_raw\compTag_189-key_9.htm 1 1
data_raw\compTag_156-key_0.htm 10 10
data_raw\compTag_156-key_1.htm 10 10
data_raw\compTag_156-key_2.htm 10 10
data_raw\compTag_156-key_3.htm 10 10
data_raw\compTag_156-key_4.htm 10 10
data_raw\compTag_156-key_5.htm 10 10
data_raw\compTag_156-key_6.htm 10 10
data_raw\compTag_156-key_7.htm 10 10
data_raw\compTag_15

In [183]:
dfqpp.query('key=="全栈设计"')

tag  tag_display tag_code   key  keyi  \
77   compTag  2018互联网300强      182  全栈设计     8   
124  compTag    AI创新成长50强      189  全栈设计     8   
206  compTag         上市公司      156  全栈设计     8   
283  compTag       中国500强      155  全栈设计     8   
324  compTag      制造业500强      186  全栈设计     8   
399  compTag          独角兽      130  全栈设计     8   

                                               payload  \
77   {'init': ['-1'], 'compTag': ['182'], 'ckid': [...   
124  {'init': ['-1'], 'compTag': ['189'], 'ckid': [...   
206  {'init': ['-1'], 'compTag': ['156'], 'ckid': [...   
283  {'init': ['-1'], 'compTag': ['155'], 'ckid': [...   
324  {'init': ['-1'], 'compTag': ['186'], 'ckid': [...   
399  {'init': ['-1'], 'compTag': ['130'], 'ckid': [...   

                         fn                                  fn_full  \
77   compTag_182-key_8-pp_0  compTag_182_2018互联网300强-key_8_全栈设计-pp_0   
124  compTag_189-key_8-pp_0    compTag_189_AI创新成长50强-key_8_全栈设计-pp_0   
206  compTag_156-key_8-pp_0         compTag_156_上市公司-key_8_全栈设计-pp_0   
283  compTag_155-key_8-pp_0       compTag_155_中国500强-key_8_全栈设计-pp_0   
324  compTag_186-key_8-pp_0      compTag_186_制造业500强-key_8_全栈设计-pp_0   
399  compTag_130-key_8-pp_0          compTag_130_独角兽-key_8_全栈设计-pp_0   

                                                fn_tsv  \
77   data_sets\compTag_182_2018互联网300强-key_8_全栈设计-p...   
124  data_sets\compTag_189_AI创新成长50强-key_8_全栈设计-pp_...   
206     data_sets\compTag_156_上市公司-key_8_全栈设计-pp_0.tsv   
283   data_sets\compTag_155_中国500强-key_8_全栈设计-pp_0.tsv   
324  data_sets\compTag_186_制造业500强-key_8_全栈设计-pp_0.tsv   
399      data_sets\compTag_130_独角兽-key_8_全栈设计-pp_0.tsv   

                                  fn_htm curPage pp   i  
77   data_raw\compTag_182-key_8-pp_0.htm       0  0   8  
124  data_raw\compTag_189-key_8-pp_0.htm       0  0  18  
206  data_raw\compTag_156-key_8-pp_0.htm       0  0  28  
283  data_raw\compTag_155-key_8-pp_0.htm       0  0  38  
324  data_raw\compTag_186-key_8-pp_0.htm       0  0  48  
399  data_raw\compTag_130-key_8-pp_0.htm       0  0  58

In [184]:
dfqpp.groupby(by=["tag_display","key"]).agg({"fn":"count"}) # .sort_values(by="fn", ascending=True)

fn
tag_display key      
2018互联网300强 UI设计   10
            交互设计   10
            体验设计   10
            信息设计    7
            全栈设计    1
            全链路设计   1
            服务设计   10
            用户体验   10
            界面设计   10
            视觉设计   10
AI创新成长50强   UI设计    5
            交互设计   10
            体验设计   10
            信息设计    1
            全栈设计    1
            全链路设计   1
            服务设计    1
            用户体验   10
            界面设计    5
            视觉设计    3
上市公司        UI设计   10
            交互设计   10
            体验设计   10
            信息设计   10
            全栈设计    1
            全链路设计   1
            服务设计   10
            用户体验   10
            界面设计   10
            视觉设计   10
中国500强      UI设计   10
            交互设计   10
            体验设计   10
            信息设计    5
            全栈设计    1
            全链路设计   1
            服务设计   10
            用户体验   10
            界面设计   10
            视觉设计   10
制造业500强     UI设计    3
            交互设计    9
            体验设计    9
            信息设计    1
            全栈设计    1
            全链路设计   1
            服务设计    1
            用户体验    9
            界面设计    3
            视觉设计    4
独角兽         UI设计   10
            交互设计   10
            体验设计   10
            信息设计    3
            全栈设计    1
            全链路设计   1
            服务设计   10
            用户体验   10
            界面设计   10
            视觉设计   10

In [185]:
dfqpp.query('key=="全栈设计"').index

Int64Index([77, 124, 206, 283, 324, 399], dtype='int64')

In [187]:
# 用查询框dfqpppp批量抓取, tsv raw的暂存, 并把抓取状态及大心存入dfqppppr, 和dfqpppp一一对映i
records_crawl = []

url = r.url

for i in dfqpp.index: #dfqpp.query('key=="全栈设计"').index: #list (dfqpp.index)[0:1]: #dfqpp.index:
    payload = dfqpp.loc[i]['payload']
    fn_tsv = dfqpp.loc[i]['fn_tsv']
    fn_htm = dfqpp.loc[i]['fn_htm']
    key = dfqpp.loc[i]['key']
    tag_display = dfqpp.loc[i]['tag_display']
    curPage = int(dfqpp.loc[i]['curPage'])# 加 curPage
    
    pp = int(dfqpp.loc[i]['pp'])
    
    df_tsv, htm_raw =  requests_liepin(url, payload)   # 抓
    
    df_tsv.index.name = "rank"
    df_tsv = df_tsv.reset_index()
    df_tsv = df_tsv.assign (i = i)
    df_tsv = df_tsv.assign (rank = lambda x: x["rank"] +1)
    df_tsv = df_tsv.assign (tag_display = tag_display)  # 区分  tag_display    
    df_tsv = df_tsv.assign (keyword = key)              # 区分  keyword    
    df_tsv = df_tsv.assign (curPage = curPage)          # 区分  curPage    

        
    df_tsv.to_csv(fn_tsv, encoding="utf8", sep="\t")  # 存 tsv
    with open(fn_htm, "wb") as f:                     # 存 htm 
        f.write(htm_raw)

    stats_code = r.status_code # http code，记录检查用
    size_tsv = len(df_tsv)  # tsv大小，记录检查用
    size_htm = len(htm_raw) # htm大小，记录检查用
    
    _record_ = {"i":i, "pp":pp, "size_tsv":size_tsv, "size_htm":size_htm, "stats_code":stats_code}
    print ('抓',pp, tag_display,key,_record_)
    records_crawl.append(_record_)
    
    time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
    
dfqppr = pd.DataFrame(records_crawl)

抓 0 2018互联网300强 界面设计 {'i': 0, 'pp': 0, 'size_tsv': 40, 'size_htm': 226525}
抓 1 2018互联网300强 界面设计 {'i': 1, 'pp': 1, 'size_tsv': 40, 'size_htm': 236859}
抓 2 2018互联网300强 界面设计 {'i': 2, 'pp': 2, 'size_tsv': 40, 'size_htm': 235898}
抓 3 2018互联网300强 界面设计 {'i': 3, 'pp': 3, 'size_tsv': 40, 'size_htm': 233224}
抓 4 2018互联网300强 界面设计 {'i': 4, 'pp': 4, 'size_tsv': 40, 'size_htm': 231596}
抓 5 2018互联网300强 界面设计 {'i': 5, 'pp': 5, 'size_tsv': 40, 'size_htm': 224924}
抓 6 2018互联网300强 界面设计 {'i': 6, 'pp': 6, 'size_tsv': 40, 'size_htm': 224863}
抓 7 2018互联网300强 界面设计 {'i': 7, 'pp': 7, 'size_tsv': 40, 'size_htm': 225323}
抓 8 2018互联网300强 界面设计 {'i': 8, 'pp': 8, 'size_tsv': 40, 'size_htm': 235356}
抓 9 2018互联网300强 界面设计 {'i': 9, 'pp': 9, 'size_tsv': 40, 'size_htm': 240508}
抓 0 2018互联网300强 UI设计 {'i': 10, 'pp': 0, 'size_tsv': 40, 'size_htm': 224064}
抓 1 2018互联网300强 UI设计 {'i': 11, 'pp': 1, 'size_tsv': 40, 'size_htm': 235032}
抓 2 2018互联网300强 UI设计 {'i': 12, 'pp': 2, 'size_tsv': 40, 'size_htm': 233764}
抓 3 2018互联网300强 UI设计 {

抓 7 AI创新成长50强 用户体验 {'i': 109, 'pp': 7, 'size_tsv': 40, 'size_htm': 244123}
抓 8 AI创新成长50强 用户体验 {'i': 110, 'pp': 8, 'size_tsv': 40, 'size_htm': 241686}
抓 9 AI创新成长50强 用户体验 {'i': 111, 'pp': 9, 'size_tsv': 40, 'size_htm': 241310}
抓 0 AI创新成长50强 体验设计 {'i': 112, 'pp': 0, 'size_tsv': 40, 'size_htm': 239441}
抓 1 AI创新成长50强 体验设计 {'i': 113, 'pp': 1, 'size_tsv': 40, 'size_htm': 243908}
抓 2 AI创新成长50强 体验设计 {'i': 114, 'pp': 2, 'size_tsv': 40, 'size_htm': 238752}
抓 3 AI创新成长50强 体验设计 {'i': 115, 'pp': 3, 'size_tsv': 40, 'size_htm': 242279}
抓 4 AI创新成长50强 体验设计 {'i': 116, 'pp': 4, 'size_tsv': 40, 'size_htm': 240883}
抓 5 AI创新成长50强 体验设计 {'i': 117, 'pp': 5, 'size_tsv': 40, 'size_htm': 240131}
抓 6 AI创新成长50强 体验设计 {'i': 118, 'pp': 6, 'size_tsv': 40, 'size_htm': 243219}
抓 7 AI创新成长50强 体验设计 {'i': 119, 'pp': 7, 'size_tsv': 40, 'size_htm': 244017}
抓 8 AI创新成长50强 体验设计 {'i': 120, 'pp': 8, 'size_tsv': 40, 'size_htm': 241776}
抓 9 AI创新成长50强 体验设计 {'i': 121, 'pp': 9, 'size_tsv': 40, 'size_htm': 241261}
抓 0 AI创新成长50强 服务设计 {'i': 

抓 7 中国500强 UI设计 {'i': 225, 'pp': 7, 'size_tsv': 40, 'size_htm': 229223}
抓 8 中国500强 UI设计 {'i': 226, 'pp': 8, 'size_tsv': 40, 'size_htm': 228832}
抓 9 中国500强 UI设计 {'i': 227, 'pp': 9, 'size_tsv': 40, 'size_htm': 230001}
抓 0 中国500强 视觉设计 {'i': 228, 'pp': 0, 'size_tsv': 40, 'size_htm': 223576}
抓 1 中国500强 视觉设计 {'i': 229, 'pp': 1, 'size_tsv': 40, 'size_htm': 230348}
抓 2 中国500强 视觉设计 {'i': 230, 'pp': 2, 'size_tsv': 40, 'size_htm': 233014}
抓 3 中国500强 视觉设计 {'i': 231, 'pp': 3, 'size_tsv': 40, 'size_htm': 232722}
抓 4 中国500强 视觉设计 {'i': 232, 'pp': 4, 'size_tsv': 40, 'size_htm': 226916}
抓 5 中国500强 视觉设计 {'i': 233, 'pp': 5, 'size_tsv': 40, 'size_htm': 241738}
抓 6 中国500强 视觉设计 {'i': 234, 'pp': 6, 'size_tsv': 40, 'size_htm': 231183}
抓 7 中国500强 视觉设计 {'i': 235, 'pp': 7, 'size_tsv': 40, 'size_htm': 224282}
抓 8 中国500强 视觉设计 {'i': 236, 'pp': 8, 'size_tsv': 40, 'size_htm': 231072}
抓 9 中国500强 视觉设计 {'i': 237, 'pp': 9, 'size_tsv': 40, 'size_htm': 233933}
抓 0 中国500强 交互设计 {'i': 238, 'pp': 0, 'size_tsv': 40, 'size_htm': 

抓 3 独角兽 UI设计 {'i': 339, 'pp': 3, 'size_tsv': 40, 'size_htm': 239093}
抓 4 独角兽 UI设计 {'i': 340, 'pp': 4, 'size_tsv': 40, 'size_htm': 237821}
抓 5 独角兽 UI设计 {'i': 341, 'pp': 5, 'size_tsv': 40, 'size_htm': 240227}
抓 6 独角兽 UI设计 {'i': 342, 'pp': 6, 'size_tsv': 40, 'size_htm': 237622}
抓 7 独角兽 UI设计 {'i': 343, 'pp': 7, 'size_tsv': 40, 'size_htm': 237208}
抓 8 独角兽 UI设计 {'i': 344, 'pp': 8, 'size_tsv': 40, 'size_htm': 239596}
抓 9 独角兽 UI设计 {'i': 345, 'pp': 9, 'size_tsv': 40, 'size_htm': 237059}
抓 0 独角兽 视觉设计 {'i': 346, 'pp': 0, 'size_tsv': 40, 'size_htm': 240003}
抓 1 独角兽 视觉设计 {'i': 347, 'pp': 1, 'size_tsv': 40, 'size_htm': 243170}
抓 2 独角兽 视觉设计 {'i': 348, 'pp': 2, 'size_tsv': 40, 'size_htm': 241872}
抓 3 独角兽 视觉设计 {'i': 349, 'pp': 3, 'size_tsv': 40, 'size_htm': 240469}
抓 4 独角兽 视觉设计 {'i': 350, 'pp': 4, 'size_tsv': 40, 'size_htm': 241488}
抓 5 独角兽 视觉设计 {'i': 351, 'pp': 5, 'size_tsv': 40, 'size_htm': 242260}
抓 6 独角兽 视觉设计 {'i': 352, 'pp': 6, 'size_tsv': 40, 'size_htm': 240218}
抓 7 独角兽 视觉设计 {'i': 353, 'pp': 7, '

In [417]:
dfqppr

i  pp  size_tsv  size_htm
0      0   0        40    226525
1      1   1        40    236859
2      2   2        40    235898
3      3   3        40    233224
4      4   4        40    231596
..   ...  ..       ...       ...
396  396   0        40    239595
397  397   1        40    234766
398  398   2        40    239871
399  399   0         3    106762
400  400   0        18    151193

[401 rows x 4 columns]

### 翻页整合

In [360]:
all_fn_tsv = [x for x in  dfqpp.fn_tsv.to_list() if "-pp_" in x]

In [419]:

# 汇总所有查询结果页
_temp_ = list()
for filename in all_fn_tsv:
    df_one = pd.read_csv(filename, encoding="utf8", sep='\t', index_col=0)
    _temp_.append(df_one)
dfl = pd.concat(_temp_).reset_index(drop=True)
dfl.index.name='i'
dfl

rank    edu     经验          薪水           时间                职称    公司地点  \
i                                                                             
0        1  本科及以上   1-3年          面议  2020年04月25日       29911-UI工程师      北京   
1        2  本科及以上   3-5年          面议  2020年04月25日       24012-UI设计师      深圳   
2        3  本科及以上  5-10年          面议  2020年04月25日      18302-UI设计组长      深圳   
3        4  本科及以上   3-5年          面议  2020年04月25日     15618-UI视觉设计师      上海   
4        5  本科及以上   3-5年          面议  2020年04月25日  15570-游戏UI设计（深圳）      深圳   
...    ...    ...    ...         ...          ...               ...     ...   
15464   14   统招本科   3-5年  15-25k·12薪  2019年12月23日   用户体验设计师(J10487)  深圳-科技园   
15465   15  本科及以上   3-5年  40-70k·15薪  2019年08月07日              Java      北京   
15466   16  本科及以上   3-5年  50-80k·15薪  2019年08月07日                全栈      北京   
15467   17  本科及以上   3-5年  15-26k·12薪  2019年03月29日           光电互连工程师      深圳   
15468   18  本科及以上   3-5年  15-26k·12薪  2019年02月15日           光电互连工程师  东莞-松山湖   

       公司名称                                           链结  \
i                                                          
0        腾讯  https://www.liepin.com/job/1927757849.shtml   
1        腾讯  https://www.liepin.com/job/1924517327.shtml   
2        腾讯  https://www.liepin.com/job/1927414895.shtml   
3        腾讯  https://www.liepin.com/job/1927414925.shtml   
4        腾讯  https://www.liepin.com/job/1927141545.shtml   
...     ...                                          ...   
15464   房多多  https://www.liepin.com/job/1923654379.shtml   
15465  今日头条  https://www.liepin.com/job/1922317897.shtml   
15466  今日头条  https://www.liepin.com/job/1922317891.shtml   
15467    华为  https://www.liepin.com/job/1917526569.shtml   
15468    华为  https://www.liepin.com/job/1917526571.shtml   

                                         公司URL    i  tag_display keyword  \
i                                                                          
0      https://www.liepin.com/company/7983148/    0  2018互联网300强    界面设计   
1      https://www.liepin.com/company/7983148/    0  2018互联网300强    界面设计   
2      https://www.liepin.com/company/7983148/    0  2018互联网300强    界面设计   
3      https://www.liepin.com/company/7983148/    0  2018互联网300强    界面设计   
4      https://www.liepin.com/company/7983148/    0  2018互联网300强    界面设计   
...                                        ...  ...          ...     ...   
15464  https://www.liepin.com/company/7871137/  400          独角兽   全链路设计   
15465  https://www.liepin.com/company/9352611/  400          独角兽   全链路设计   
15466  https://www.liepin.com/company/9352611/  400          独角兽   全链路设计   
15467   https://www.liepin.com/company/954482/  400          独角兽   全链路设计   
15468   https://www.liepin.com/company/954482/  400          独角兽   全链路设计   

      curPage  
i              
0           0  
1           0  
2           0  
3           0  
4           0  
...       ...  
15464       0  
15465       0  
15466       0  
15467       0  
15468       0  

[15469 rows x 14 columns]

In [421]:
dfl.to_csv(fn["output"]["dfl"], encoding="utf8", sep="\t")

In [363]:
dfl.groupby(by=["tag_display","keyword"]).count()

rank  edu   经验   薪水   时间   职称  公司地点  公司名称   链结  公司URL  \
tag_display keyword                                                          
2018互联网300强 UI设计      400  400  400  400  400  400   358   400  400    400   
            交互设计      400  400  400  400  400  400   347   400  400    400   
            体验设计      400  400  400  400  400  400   336   400  400    400   
            信息设计      253  253  253  253  253  253   219   253  253    253   
            全栈设计        8    8    8    8    8    8     8     8    8      8   
            全链路设计      40   40   40   40   40   40    40    40   40     40   
            服务设计      400  400  400  400  400  400   370   400  400    400   
            用户体验      400  400  400  400  400  400   333   400  400    400   
            界面设计      400  400  400  400  400  400   356   400  400    400   
            视觉设计      400  400  400  400  400  400   353   400  400    400   
AI创新成长50强   UI设计      166  166  166  166  166  166   146   166  166    166   
            交互设计      400  400  400  400  400  400   357   400  400    400   
            体验设计      400  400  400  400  400  400   357   400  400    400   
            信息设计       39   39   39   39   39   39    34    39   39     39   
            全链路设计       2    2    2    2    2    2     2     2    2      2   
            服务设计       40   40   40   40   40   40    33    40   40     40   
            用户体验      400  400  400  400  400  400   357   400  400    400   
            界面设计      166  166  166  166  166  166   146   166  166    166   
            视觉设计      107  107  107  107  107  107    93   107  107    107   
上市公司        UI设计      400  400  400  400  400  400   357   400  400    400   
            交互设计      400  400  400  400  400  400   340   400  400    400   
            体验设计      400  400  400  400  400  400   331   400  400    400   
            信息设计      374  374  374  374  374  374   327   374  374    374   
            全栈设计        5    5    5    5    5    5     5     5    5      5   
            全链路设计      40   40   40   40   40   40    38    40   40     40   
            服务设计      400  400  400  400  400  400   352   400  400    400   
            用户体验      400  400  400  400  400  400   332   400  400    400   
            界面设计      400  400  400  400  400  400   358   400  400    400   
            视觉设计      400  400  400  400  400  400   346   400  400    400   
中国500强      UI设计      400  400  400  400  400  400   376   400  400    400   
            交互设计      400  400  400  400  400  400   374   400  400    400   
            体验设计      400  400  400  400  400  400   374   400  400    400   
            信息设计      182  182  182  182  182  182   163   182  182    182   
            全栈设计        4    4    4    4    4    4     4     4    4      4   
            全链路设计      40   40   40   40   40   40    40    40   40     40   
            服务设计      400  400  400  400  400  400   393   400  400    400   
            用户体验      400  400  400  400  400  400   374   400  400    400   
            界面设计      400  400  400  400  400  400   377   400  400    400   
            视觉设计      400  400  400  400  400  400   378   400  400    400   
制造业500强     UI设计      117  117  117  117  117  117    81   117  117    117   
            交互设计      335  335  335  335  335  335   230   335  335    335   
            体验设计      335  335  335  335  335  335   230   335  335    335   
            信息设计       24   24   24   24   24   24    21    24   24     24   
            服务设计       40   40   40   40   40   40    35    40   40     40   
            用户体验      335  335  335  335  335  335   230   335  335    335   
            界面设计      117  117  117  117  117  117    81   117  117    117   
            视觉设计      159  159  159  159  159  159   139   159  159    159   
独角兽         UI设计      400  400  400  400  400  400   343   400  400    400   
            交互设计      400  400  400  400  400  400   326   400  400    400   
            体验设计      400  400  400  400  4

In [364]:
len(dfl)

15469

In [365]:
dfl.columns

Index(['rank', 'edu', '经验', '薪水', '时间', '职称', '公司地点', '公司名称', '链结', '公司URL',
       'i', 'tag_display', 'keyword', 'curPage'],
      dtype='object')

In [366]:
dfl.nunique()

rank             40
edu               6
经验                6
薪水              453
时间              393
职称             3646
公司地点            358
公司名称            368
链结             4763
公司URL           372
i               398
tag_display       6
keyword          10
curPage          10
dtype: int64

## 生产用代码C

In [369]:
df_content = pd.DataFrame({"href":sorted(list(dfl.链结.unique()))})
df_content = df_content.assign ( sn = lambda x: [_x_.split("/job/")[1].split(".")[0] for _x_ in x["href"]])
df_content = df_content.assign ( fn = lambda x: [fn["output"]["path_raw_content"].format(sn=_x_) for _x_ in x["sn"]])

In [370]:
df_content.loc[0:1]

href          sn  \
0  https://www.liepin.com/job/1910033980.shtml  1910033980   
1  https://www.liepin.com/job/1910072901.shtml  1910072901   

                                fn  
0  data_raw_content/1910033980.htm  
1  data_raw_content/1910072901.htm

In [387]:
'''
dfcr =pd.read_csv("dfcr.tsv", encoding="utf8", sep="\t", index_col=0) # 读入
dfcr
'''
dfcr.to_csv(fn["output"]["dfcr"], encoding="utf8", sep="\t") # 写入
dfcr


i          sn  stats_code  size_htm
0        0  1910033980         200     97392
1        1  1910072901         200     94011
2        2  1910088706         200     97439
3        3  1910586808         200     98541
4        4  1910586882         200     98951
...    ...         ...         ...       ...
4758  4758   199648265         200     96768
4759  4759   199682147         200     92277
4760  4760   199789518         200     96090
4761  4761   199812297         200     97840
4762  4762   199912913         200     97195

[4763 rows x 4 columns]

## 生产用代码C

In [413]:
# requests_liepin查询首页存放与解析
def requests_liepin_content( html ):
    数据= pd.DataFrame()
    
    主要元素 = html.xpath( '//div[@class="about-position"]', first=True)
        
    # 该职位已暂停招聘    
    if 主要元素 is None:
        X= html.xpath('//div[@class="job-offline-container"]/p/text()', first=True)
        数据字典 = {x:X for x in ['title_info', 'basic_location', 'basic_time', 'job_desc', 'job-qualifications', 'comp-tag-list', 'comp-tag-list-data', 'misc', 'i', 'sn'] }
        数据 = pd.DataFrame([数据字典])
        return (数据)
    
    dict_xpaths={ 
        'text_simple': {
            'title_info': '//div[contains(@class,"title-info")]/h1',
            'basic_location': '//p[@class="basic-infor"]/span/a',
            'basic_time': '//p[@class="basic-infor"]/time/@title',
            'job_desc': '//div[contains(@class,"job-description")]/div',
        },
        'text': {
        },
        'list_complex': {
            'misc': '//h3[text()[contains(.,"其他信息：")]]//following-sibling::div/ul/li',
        },
        'list': {
            'job-qualifications': '//div[contains(@class,"job-qualifications")]/span/text()',
            'comp-tag-list': '//ul[contains(@class,"comp-tag-list")]/li/span/text()',
            'comp-tag-list-data': '//ul[contains(@class,"comp-tag-list")]/li[@data-title]/@data-title',
        }
    }

    def get_e_text(_xpath_):
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_text_simple(_xpath_):
        try:
            暂存结果 = 主要元素.xpath(_xpath_, first=True).text
        except:
            暂存结果 = 主要元素.xpath(_xpath_, first=True)
        return(暂存结果)

    def get_e_list_complex(_xpath_):
        X = 主要元素.xpath(_xpath_)
        暂存结果 = {x.xpath('//span/text()', first=True): x.xpath('//label/text()', first=True) for x in X}
        return(暂存结果)

    def get_e_list(_xpath_):
        暂存结果 = 主要元素.xpath(_xpath_)
        return(暂存结果)

    数据字典 = dict()
    数据字典 = {k:get_e_text_simple(v) for k,v in dict_xpaths['text_simple'].items()}

    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_list(v) for k,v in dict_xpaths['list'].items()})
    数据字典.update({k:get_e_list_complex(v) for k,v in dict_xpaths['list_complex'].items()})

    数据 = pd.DataFrame([数据字典])
    
    return (数据) #return (数据字典, 主要元素)

In [415]:
def read_fn_htm (fn_htm):
    with open (fn_htm, encoding = "utf8", mode = "r") as fp:
        html_load = fp.read()                  #html_load
    html = requests_html.HTML(html= html_load, url='https://localhost/')
    return(html)

d_list = []
for i, fn_htm in df_content['fn'].to_dict().items(): # df_content.loc[966:]['fn'].to_dict().items(): #df_content.loc[200:503]['fn'].to_dict().items()
    html = read_fn_htm(fn_htm)
    sn = df_content.loc[i]['sn']

    d = requests_liepin_content(html)
    d["i"]=i
    d["sn"]=sn
    d_list.append(d.set_index(["i"]))
    
df_content_page = pd.concat(d_list)
df_content_page.to_csv(fn["output"]["df_content_page"], encoding="utf8", sep="\t")

In [416]:
df_content_page

title_info basic_location   basic_time  \
i                                                  
0            UI界面设计师             成都  2020年02月26日   
1               产品经理         重庆-石桥铺  2020年04月16日   
2            互联网产品经理          上海-梅陇  2019年12月20日   
3         相机APP开发工程师          北京-清河  2020年03月27日   
4     Touch BSP软件工程师          北京-清河  2020年03月27日   
...              ...            ...          ...   
4758         高级平面设计师          深圳-坂田  2020年04月22日   
4759           平面设计师         深圳-南山区  2019年08月22日   
4760         软件UI设计师             广州  2020年02月26日   
4761     政企大客户经理（KA）         广州-东风东  2020年04月24日   
4762           视觉设计师             广州  2020年02月26日   

                                               job_desc  \
i                                                         
0     【薪资待遇】面议\n【工作职责】\n1.负责游戏界面及相关UI的美术设计、创意提案和制作工作...   
1     岗位职责：\n1.结合运营或营销部门需求完成前期产品市场需求分析、竞品市场调研及产品可行性分...   
2     职位描述：\n\n1.负责PC及移动端软件的产品规划、管理工作\n\n2.负责PC及移动端产...   
3     岗位职责：\n1、实现UI以及相关逻辑功能，完善用户体验；\n2、优化相机各项性能指标；\n...   
4     岗位职责：\n负责Android手机touch相关的底层软件开发\n负责手机产品的touch...   
...                                                 ...   
4758  岗位职责：\n1.负责公司各类活动项目中的平面设计工作，如：主视觉、延展物料、邀请函、画册等...   
4759  工作职责\n1.房地产项目营销策划工作（形象推广，品牌推广，宣传活动的相关平面设计工作）；\...   
4760  【薪资待遇】面议\n【工作职责】\n按需求设计制作符合软件产品功能、风格的界面及相关UI。\...   
4761  1. 根据公司发展的战略目标，负责政府、国企等重点大客户服务项目的设计、谈判、实施；\n2....   
4762  【薪资待遇】10万/年-20万/年\n【工作职责】\n1、网页设计方向：\n（1）负责网站整...   

               job-qualifications  \
i                                   
0         [学历不限, 经验不限, 普通话, 年龄不限]   
1       [本科及以上, 3-5年, 语言不限, 年龄不限]   
2       [本科及以上, 1-3年, 语言不限, 年龄不限]   
3       [本科及以上, 经验不限, 语言不限, 年龄不限]   
4       [本科及以上, 经验不限, 语言不限, 年龄不限]   
...                           ...   
4758    [大专及以上, 3-5年, 语言不限, 年龄不限]   
4759   [本科及以上, 5-10年, 语言不限, 年龄不限]   
4760      [学历不限, 经验不限, 普通话, 年龄不限]   
4761  [大专及以上, 经验不限, 语言不限, 25-35岁]   
4762     [学历不限, 经验不限, 语言不限, 年龄不限]   

                                          comp-tag-list  \
i                                                         
0                                [2018互联网300强, 互联网百强企业]   
1                                 [制造业500强, 五险一金, 绩效奖金]   
2     [独角兽, 毕马威中国汽车科技50强, 发展空间大, 岗位晋升, 团队聚餐, 节日礼物, 公...   
3     [2018福布斯中国创新企业, 中国500强, 上市公司, 2018中国出海品牌30强, 2...   
4     [2018福布斯中国创新企业, 中国500强, 上市公司, 2018中国出海品牌30强, 2...   
...                                                 ...   
4758  [独角兽, 2018中国出海品牌30强, 2018互联网300强, 500强, 管理规范, ...   
4759   [定期体检, 五险一金, 500强, 带薪年假, 公司规模大, 领导好, 岗位晋升, 扁平管理]   
4760                             [2018互联网300强, 互联网百强企业]   
4761  [上市公司, 绩效奖金, 五险一金, 扁平管理, 领导好, 定期体检, 岗位晋升, 股票期权...   
4762                                [2018互联网300强, 五险一金]   

                                     comp-tag-list-data  \
i                                                         
0                   [由中科院主管、科学出版社主办的《互联网周刊》于2018年评选。, ]   
1                            [由中国企业联合会、中国企业家协会联合发布, , ]   
2     [成立十年以内，估值十亿美金以上的未上市公司。, 顶级会计师事务所毕马威发布“汽车科技企业5...   
3     [福布斯杂志评选的，通过技术创新、模式创新引领行业发展或对行业进步产生重大影响力的中国企业。...   
4     [福布斯杂志评选的，通过技术创新、模式创新引领行业发展或对行业进步产生重大影响力的中国企业。...   
...                                                 ...   
4758  [成立十年以内，估值十亿美金以上的未上市公司。, , 由中科院主管、科学出版社主办的《互联网...   
4759                                   [, , , , , , , ]   
4760                [由中科院主管、科学出版社主办的《互联网周刊》于2018年评选。, ]   
4761                               [, , , , , , , , , ]   
4762                [由中科院主管、科学出版社主办的《互联网周刊》于2018年评选。, ]   

                                                   misc          sn  
i                                                                    
0     {'所属部门：': '美术部', '专业要求：': '不限', '汇报对象：': '无', ...  1910033980  
1     {'所属部门：': '宗申创新平台', '专业要求：': '不限', '汇报对象：': '无...  1910072901  
2     {'所属部门：': '研发部', '专业要求：': '不限', '汇报对象：': '无', ...  1910088706  
3     {'所属部门：': '手机', '专业要求：': '不限', '汇报对象：': '无', '...  1910586808  
4     {'所属部门：': '手机', '专业要求：': '不限', '汇报对象：': '无', '...  1910586882  
...                                                 ...  

In [ ]:
with pd.ExcelWriter(fn["output"]["datasets_xls"]) as writer:    
    dfl.to_excel(writer,sheet_name="result") 
    先行再国.to_excel(writer,sheet_name="先行再国") 

-----


测试用代码


-----

## 测试用代码C

In [393]:
print (i,len(d_list), sn)

966 966 1919836993


In [239]:
# one at a time
fn_htm = df_content.loc[500]['fn']
print (fn_htm)
with open (fn_htm, encoding = "utf8", mode = "r") as fp:
        html_load = fp.read()                  #html_load
html = requests_html.HTML(html= html_load, url='https://localhost/')
html

data_raw_content/1917222603.htm


<HTML url='https://localhost/'>

In [349]:
df_content.loc[500:503]['fn'].to_dict()

{500: 'data_raw_content/1917222603.htm',
 501: 'data_raw_content/1917223853.htm',
 502: 'data_raw_content/1917224803.htm',
 503: 'data_raw_content/1917226409.htm'}

In [399]:
# 该职位已暂停招聘
X= html.xpath('//div[@class="job-offline-container"]/p/text()', first=True)
X

'该职位已暂停招聘'

In [395]:
# one at a time
X= html.xpath('//div[@class="about-position"]', first=True) \
     .xpath('//div[@class="job-offline-container"]', first=True)
    #.xpath('//p[@class="basic-infor"]/time/@title', first=True)
    #   .xpath('//h3[text()[contains(.,"其他信息：")]]//following-sibling::div/ul/li')
    #.xpath('//p[@class="basic-infor"]/span/a')
    #.xpath('//ul[contains(@class,"comp-tag-list")]/li[@data-title]/@data-title')
    #.xpath('//div[contains(@class,"job-qualifications")]/span/text()')
    #.xpath('//div[contains(@class,"title-info")]/h1', first=True).text
    
#[x.xpath('//span/text()', first=True) for x in X]
#[x.xpath('//label/text()', first=True) for x in X]
# {x.xpath('//span/text()', first=True): x.xpath('//label/text()', first=True) for x in X}
X

AttributeError: 'NoneType' object has no attribute 'xpath'

In [342]:
d, main = requests_liepin_content(html)
display(d)
print (main)

title_info basic_location   basic_time  \
0  高级前端开发工程师         北京-海淀区  2019年01月23日   

                                            job_desc  \
0  职责描述：\n1、负责前端技术选型、架构开发\n2、依据产品需求与UE设计师、后台工程师协同...   

          job-qualifications  \
0  [本科及以上, 经验不限, 语言不限, 年龄不限]   

                                       comp-tag-list  \
0  [独角兽, 2018全球人工智能百强, 2018互联网300强, 2018中国创新成长百强,...   

                                  comp-tag-list-data  \
0  [成立十年以内，估值十亿美金以上的未上市公司。, 全球顶尖的风险投资调研机构CB Insig...   

                                                misc  
0  {'所属部门：': '系统平台', '专业要求：': '不限', '汇报对象：': '无',...

<Element 'div' class=('about-position',)>


In [265]:
main.xpath('//div[contains(@class,"job-description")]/div', first=True).text

'职责描述：\n1、负责前端技术选型、架构开发\n2、依据产品需求与UE设计师、后台工程师协同工作，完成高质量的Web前端开发\n3、快速响应产品迭代以及优化升级，持续优化前端体验，提升友好和易用性\n\n任职要求：\n1、本科及以上学历，计算机、通信等相关专业，5年以上前端开发经验；\n2、熟练掌握前端相关技术（HTML5/CSS3/Javascript），理解Web标准，对前端性能优化、模块化开发等有深入了解，对主流前端框架（ React \\ Vue \\ Angular 等）至少一种有深入应用并深入理解其设计原理；\n3、熟练掌握nodejs、gulp、bower等常用开发工具；\n4、具有较强的沟通能力，分析和解决问题的能力，良好的团队协作能力；\n5、热爱前端技术，有强烈的求知欲、和进取心 ，能及时关注和学习业界最新的前端技术。'

In [ ]:
_df_ = requests_liepin_content(html_load)   #html_load